# 12. Manipulation de séries temporelles

Ce notebook permet de découvrir la manipulation des données temporelles avec l'écosystème pandas. Nous découvrirons quelques problématiques courantes lors de la manipulation de données temporelles (changement d'échelle de dates, enrichissement de données, sélection de données, ...).

Le dataset utilisé correspond au trafic automobile sur une route comportant plusieurs voies de circulation. Les trafics des différentes voies ont été agrégés en une valeur `flow`. L'attribut `velocity` indique la vitesse moyenne des véhicules sur toutes les voies. Deux colonnes indiquent si la mesure a été prise pendant un jour férié ou pendant une période de vacance scolaire.

In [ ]:
import plotly
plotly.__version__

In [ ]:
import plotly.express as px

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import plotly as py
import plotly.graph_objs as go
py.offline.init_notebook_mode(connected=True)

## 12.1 Chargement du dataset

Chargez le fichier `data/traffic-data.csv` dans une dataframe, puis :
1.  Affichez les types des colonnes du dataset
2.  Visualisez les 20 premières lignes et un échantillon pour intépréter les colonnes non numériques
3.  Affichez une description statistique des variables numériques
4.  Affichez un histogramme et un boxplot pour chaque **grandeur numérique**
5.  Affichez la matrice de corrélation entre ces grandeurs numériques
6.  Testez la significativité de la corrélation de pearson entre `flow` et `velocity` avec [`scipy.stats.pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html)

In [ ]:
data = pd.read_csv()

In [ ]:
#1

In [ ]:
#2

In [ ]:
#3

In [ ]:
#4.1

In [ ]:
#4.2

In [ ]:
#5

In [ ]:
#6

## 12.2 Manipulation des dates

Il existe de nombreux formats pour les valeurs de temps (timestamps) : Timestamps numpy et datetime en particulier. Jouons un peu avec ces formats :

A l'aide du module [`time`](https://docs.python.org/3/library/time.html), définissez une variable `current_time` calculée au moment de l'exécution. Puis utilisez la méthode [`strftime`](https://docs.python.org/3/library/datetime.html#datetime.date.strftime) pour la convertir sous la forme d'une chaîne de caractère au format `YYYY-MM-DD HH:mm:ss` et stockez cette chaine dans une variable :

Convertissez cette chaîne de caractère au format `np.datetime64` :

Utilisez la méthode [`datetime.datetime.strptime`](https://docs.python.org/3/library/datetime.html#datetime.datetime.strptime) du package `datetime` pour parser la date qui est sous forme de chaîne de caractères en datetime.
*Vous devriez obtenir un resultat du type ```datetime(Y, M, D, H, m, s, ms)```*

In [ ]:
from datetime import datetime as dt

Transformez la date depuis le format numpy vers le format datetime et retrouvez le même résultat trouvé à la question précédente.

Le format de données [`datetime`](https://docs.python.org/3/library/datetime.html#datetime-objects) est le plus riche : il offre le plus de fonctionnalités, comme l'accès à certains attributs très utiles pour les dates : année, mois, semaine dans le calendrier annuel, jour de la semaine... Pour pouvoir en tirer parti, il faut convertir nos horodates en datetime. Convertissez la colonne `horodate` en datetime en utilisant la fonction [`pandas.to_datetime`](https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html) :

Ajoutez à votre dataframe une colonne pour chaque attribut :
- l'année
- le mois
- la date (YYYY-MM-DD)
- le jour de la semaine
- l'heure
- la minute

Vérifiez le résultat obtenu.

In [ ]:
data

## 12.3 Vérification de la complétion temporelle de la série

Une série temporelle possède une fréquence $F$ : toutes les $F$ secondes, une valeur doit être présente dans les données.
Il peut arriver qu'une série temporelle soit incomplète. Deux cas peuvent expliquer cela :
1. Absence d'enregistrement à une horodate donnée donnant lieu à un tuple manquant.
2. Absence de données due au changement d'horaire. En effet, les séries temporelles sont souvent exprimées sur l'échelle de temps UTC, qui est une échelle universelle. Les échelles de temps locales sont généralement décalées par rapport à UTC et la valeur de ce décalage change avec le changement d'heure hiverna/estival.

Or pour certains usages (ex. pour la prédiction de trafic), les régularités de trafic s'observent non pas en UTC, mais en temps local.

Il est donc nécessaire de convertir des données UTC en localtime, ce qui genère des trous et des redondances dans les données...

Trouvez les dates de début et de fin du dataset et calculez l'extension temporelle correspondante :

Pour convertir les datetime UTC en dates locales (CET), nous pouvons utiliser Pandas ou la librairire [pytz](https://pypi.org/project/pytz/).

Une datetime est par défaut sans timezone (timezone naive). Il faut donc dans un premier temps convertir notre datetime en timezone aware puis la convertir dans la timezone cible.

Notez la présence de la timezone dans l'affichage de la nouvelle datetime et la conversion en temps local dans la sortie du formattage de cette datetime.

In [ ]:
import pytz

mydt = dt(2018, 5, 25, 15, 12, 30)
cet_tz = pytz.timezone('CET')
mydt_utc = pytz.utc.localize(mydt)
print("Datetime format UTC:", mydt_utc)
mydt_cet = mydt_utc.astimezone(cet_tz)
print("Datetime format CET:", mydt_cet)
print("Datetime format string:", mydt.strftime("%Y-%m-%d %H:%M:%S"))
decalage = cet_tz.utcoffset(mydt)
print("Décalage entre UTC et CET (en heure):", decalage)

Créez une fonction `convert_utc_to_cet` prenant en paramètre une datetime et renvoyant cette datetime dans la timezone CET, puis créez une nouvelle colonne `localtime_horodate` dans votre dataframe en utilisant la méthode [`pandas.Series.apply`](https://pandas.pydata.org/docs/reference/api/pandas.Series.apply.html) sur la colonne `horodate` :

In [ ]:
data

Est-ce que cette méthode prend en compte les changements d'heures d'été/d'hiver ?

*Le dernier dimanche de mars 2018 correspond à 25/03/2018*

Pour rechercher les horodates manquantes dans la série temporelle indexée sur le temps local, nous pouvons générer la liste complète des horodates à la **fréquence de 6 minutes** entre le premier et le dernier horodate en temps local.

Utilisez [`pandas.date_range`](https://pandas.pydata.org/docs/reference/api/pandas.date_range.html) pour obtenir la liste des timestamps sur la plage de temps considérée, puis créez une dataframe contenant une seule colonne avec cette liste :

Réalisez une jointure avec [`pandas.merge`](https://pandas.pydata.org/docs/reference/api/pandas.merge.html) entre cette dataframe et notre dataframe contenant toutes les données.

*Faites attention à bien choisir le bon type de jointure via le paramètre ```how```. Sachant que nous souhaitons conserver la liste des dates complètes.*

Identifiez les tuples correspondant à des horodates manquantes dans le dataset et déterminez la cause probable pour ces manquants :

## 12.4 Correction des valeurs (valeurs décalées et manquantes)

Nous allons remplacer ces valeurs manquantes par la valeur pour le même jour la semaine suivante. Nous pourrions aussi recourir à la méthode `interpolate` pour réaliser une interpolation des valeurs manquantes entre deux valeurs présentes.

Commencez par appliquez la méthode [`pandas.DataFrame.shift`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.shift.html) sur les colonnes `flow` et `velocity` pour obtenir un décalage d'une semaine :

Utilisez la méthode [`pandas.DataFrame.fillna`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.fillna.html) pour remplacer les valeurs manquantes en utilisant les valeurs de la semaine suivante pour les colonnes `flow` et `velocity` :

Affichez les valeurs que nous venont d'imputer :

Recréez les colonnes indiquant l'année, le mois, la date (YYYY-MM-DD), le jour de la semaine, l'heure, la minute depuis la colonnes `localtime_horodate_complete` pour obtenir ces valeurs en timezone CET et quelles soient présentes pour toutes les lignes :

Analysez le code suivant, décrivez son fonctionnement et le résultat obtenu (pensez à utiliser des affichages intermédiaires des différentes étapes) :

In [ ]:
data_mode_per_day = data[['date', 'bank_holiday', 'school_holiday']].groupby('date').agg(pd.Series.mode).reset_index()
data_mode_per_day.date = data_mode_per_day.date.astype(str)
data.date = data.date.astype(str)
data = data.merge(
    data_mode_per_day,
    on='date',
    suffixes=('', '_imputation_value'),
    how='left'
)
for column in ['bank_holiday', 'school_holiday']:
    data[column] = data['{}_imputation_value'.format(column)]

Quelles sont les limites de la méthode d'imputation que vous avez appliqué ?

Supprimez les colonnes inutiles (`horodate`, `bank_holiday_imputation_value` et `school_holiday_imputation_value`) :

## 12.5 Identification des dates en doublon

Y a-t-il des des doublons sur l'horodate local ? S'il y en a, supprimez les.

## 12.6 Manipulation des données

Pour effectuer des analyses temporelles sur les données, il est souvent pratique d'indéxer la dataframe sur les horodates. Ré-indexez votre dataframe avec la colonne de l'horodate locale :

### 12.6.1 Analyse de la saisonnalité des données

Utilisez la fonction [`pandas.plotting.autocorrelation_plot`](https://pandas.pydata.org/docs/reference/api/pandas.plotting.autocorrelation_plot.html) pour afficher le diagramme d'autocorrélation du trafic (```flow```) et/ou utilisez la fonction `plot_autocorrelation_lags` définie ci-dessous.

Puis expliquez ce diagramme d'autocorrélation, les données sont-elles saisonnières ?

In [ ]:
from scipy import signal

def plot_autocorrelation_lags(s):
    corr = signal.correlate(s - s.mean(), s - s.mean())
    corr /= np.max(corr)
    corr = corr[len(s):]

    fig = px.scatter(corr)

    fig.show(renderer='colab')

### 12.6.2 Rééchantillonnage des données

Avec pandas, certaines opérations sur les séries temporelles sont très simplifiées, comme le rééchantillonnage.

Rééchantillonnez le dataset avec une fréquence de 12min (créez une dataframe `data_12`) et 24 min (`data_24`) en utilisant la méthode [`pandas.DataFrame.resample`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.resample.html), vous devez choisir une méthode d'aggrégation adaptée (pour les colonnes flow, occupation et velocity uniquement) :

Affichez les données de trafic (`flow`) avec la fonction [`line`](https://plotly.com/python/line-charts/)plotly express pour les 3 premiers jours de `data`, `data_12` et `data_24`. Que remarquez-vous quant à l'effet du resampling ?

### 12.6.3 Simplification des données

Simplifions les données en ne retenant que le trafic pour les mardis. Créez une dataframe `data_tuesday` ne contenant que les mardis, affichez l'autocorrélogramme du trafic et comparez le résultat obtenu avec celui obtenu sur tout le dataset :

Affichez les courbes de trafic pour tous les mardis de notre dataset avec `px.line`. Pour obtenir le meilleur affichage possible :
* appelez la méthode `update_xaxes` sur votre figure, après l'avoir créé et avant de l'afficher, en utilisant le paremètre `rangebreaks=[{'bounds': ["wed", "tues"]}]` pour cacher les sauts de dates
* utiliser le paramètre `render_mode="svg"` pour l'appel à `px.line` (pour éviter un bug connu de plotly sur l'affichage d'un graphique avec un saut de dates et le moteur de rendu WebGL)

Remarquez-vous des particularités ?

### 12.6.4 Détection des outliers

On considère mettre en œuvre un modèle prédictif linéaire pour les données de trafic du dataset. D'après le graphique ci-dessus, les données ne sont pas très qualitatives...

**Identifiez deux problèmes dans ces données.**

L'unité de détection des anomalies doit respecter la saisonnalité, soit dans cette série simplifiée, la journée.

Pour détecter (et éliminer) les jours où les données sont de mauvaise qualité, nous proposons d'utiliser la décomposition .

L'idée est la suivante :
- On effectue une décomposition
- Les valeurs de tendance, de saisonnalité et de résidu à chaque horodate peuvent être considérées comme des attributs
- On cherche les anomalies dans l'espace de ces attributs au moyen d'un algorithme d'identification d'outliers, par ex. IsolationTrees
- On devra alors rejeter les jours contenant trop d'horodates 'outlier' selon cet algorithme.

#### 12.6.4.1 Décomposition

Réalisez une décomposition du trafic avec la fonction [`statsmodels.tsa.seasonal.seasonal_decompose`](https://www.statsmodels.org/dev/generated/statsmodels.tsa.seasonal.seasonal_decompose.html) avec les paramètres suivants :
* `x=df.flow.values` : essayez de passer directement `df.flow` puis commentez
* `model='additive'` : comparez `additive` et `multiplicative`
* `period: int` : trouvez la bonne valeur
* `extrapolate_trend=True` : pour obtenir une trend sur toute la série observée

Puis affichez les valeurs des attributs résultant de la décomposition.

In [ ]:
import statsmodels
statsmodels.__version__

Affichez les graphiques de cette décomposition en utilisant la méthode `plot` sur le résultat de la décomposition. Vous pouvez afficher un graphique plus lisible en utilisant les instructions `fig.set_size_inches((16, 9))` et `fig.tight_layout()` (prenez soin d'affecter le résultat de l'appel à `plot` dans une variable nommée `fig`) :

Créez des colonnes dans votre dataframe contenant les valeurs de saisonnalité, de tendance et de résidu de notre décomposition pour chaque ligne :

#### 12.6.4.2 Recherche des outliers

Entrainez un modèle [`sklearn.ensemble.IsolationForest`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.IsolationForest.html) sur la résidu de la décomposition en précisant comme paramètre lors de la création de l'instance `contamination=.2`. Puis réalisez une prédiction sur sur ce même résidu, interprétez le résultat et affectez le à une colonne `valid_value` de votre dataframe :

Créez une series contenant la somme des `valid_value` par jour puis ajoutez une colonne (nommée `valid_value_sum`) indiquant la valeur de cette somme pour chaque ligne (indices : `groupby` et `merge` permettent de réaliser cette opération) :

Affichez un graphique des `valid_value_sum` et en dessous un graphique de la colonne `flow`, que constatez-vous ? (pour plus de lisibilité, vous pouvez réalisez une copie de votre dataframe et remplacer l'index par `df.index.astype('category')`)

On peut se fixer un seuil grossier pour décider quand une date doit être considérée comme outlier. Ajoutez une colonne `date_is_outlier` qui vaut `True` si `valid_value_sum` est inférieure à 100 :

Quels sont les jours outliers selon notre critère ?

Créez une nouvelle dataframe contenant que le trafic des jours qui ne sont pas outliers puis affichez un graphique des valeurs de `flow` pour cette dataframe :

Vérifiez que le nettoyage donne une décomposition plus encourageante...

Le nettoyage vous paraît-il satisfaisant en vue d'une prédiction par des modèles linéaires (moyenne glissante, sarima, etc.) ? Proposez (et testez !) des modifications simples.